In [ ]:
# # specify library version
# !pip install pandas==1.5.3
# !pip install numpy==1.23.5
# !pip install lightgbm==4.1.0

In [1]:
# import library
import pandas as pd
import numpy as np
import lightgbm as lgb
import pickle

In [2]:
# version check
print(pd.__version__)
print(np.__version__)
print(lgb.__version__)

1.5.3
1.23.5
4.1.0


In [3]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# change directory
%cd '/content/drive/MyDrive/Colab Notebooks/kaggle_m5/'

/content/drive/MyDrive/Colab Notebooks/kaggle_m5


In [5]:
# put csv file in input directory
# you can get datasets from
# https://www.kaggle.com/competitions/m5-forecasting-accuracy/data
train_df = pd.read_csv("input/sales_train_evaluation.csv")
prices_df = pd.read_csv("input/sell_prices.csv")
calendar_df = pd.read_csv("input/calendar.csv")

In [6]:
# sales
train_df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [7]:
# make sales data for demo
train_df = train_df[train_df['store_id'].isin(['CA_1', 'CA_2'])].reset_index(drop=True)
train_df = train_df[train_df['item_id'].str.contains('FOODS_1|FOODS_2')].reset_index(drop=True)
train_df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0,0,1,...,2,3,1,0,0,0,1,0,0,0
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,0,1,0,1,...,2,1,0,1,1,1,0,1,1,2
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,2,0,0,0,0,0,2,2,0,1
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,5,6,2,5,1,2,1,1,3,4
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,3,9,3,3,...,2,1,11,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223,FOODS_2_395_CA_2_evaluation,FOODS_2_395,FOODS_2,FOODS,CA_2,CA,0,0,0,0,...,1,0,0,0,5,0,0,0,0,0
1224,FOODS_2_396_CA_2_evaluation,FOODS_2_396,FOODS_2,FOODS,CA_2,CA,0,0,0,0,...,1,0,2,2,4,0,3,2,2,8
1225,FOODS_2_397_CA_2_evaluation,FOODS_2_397,FOODS_2,FOODS,CA_2,CA,0,0,0,0,...,0,1,1,0,0,0,0,1,0,0
1226,FOODS_2_398_CA_2_evaluation,FOODS_2_398,FOODS_2,FOODS,CA_2,CA,0,0,0,0,...,2,3,0,0,1,1,4,1,0,0


In [8]:
# set training date and prediction date
end_train_day_x = 100
predict_horizon = 7

In [9]:
# sales
index_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

sales_df = pd.melt(train_df, id_vars=index_columns, var_name='date_id', value_name='sales')
sales_df['date_id'] = sales_df['date_id'].apply(lambda x: x[2:]).astype(np.int64)

sales_df = sales_df[sales_df['date_id'] <= end_train_day_x]

display(sales_df)

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,sales
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,3
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,1,0
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,1,0
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,1,0
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,1,3
...,...,...,...,...,...,...,...,...
122795,FOODS_2_395_CA_2_evaluation,FOODS_2_395,FOODS_2,FOODS,CA_2,CA,100,0
122796,FOODS_2_396_CA_2_evaluation,FOODS_2_396,FOODS_2,FOODS,CA_2,CA,100,0
122797,FOODS_2_397_CA_2_evaluation,FOODS_2_397,FOODS_2,FOODS,CA_2,CA,100,0
122798,FOODS_2_398_CA_2_evaluation,FOODS_2_398,FOODS_2,FOODS,CA_2,CA,100,0


In [10]:
# save sales csv for demo
sales_df.to_csv('sales_demo.csv', index=False)

In [11]:
# prediction
prediction_df = pd.DataFrame()
for i in range(predict_horizon):
    temp_df = sales_df[index_columns]
    temp_df = temp_df.drop_duplicates().copy()
    temp_df['date_id'] = (end_train_day_x + i + 1)
    temp_df['sales'] = np.nan
    prediction_df = pd.concat([prediction_df, temp_df])

prediction_df.reset_index(drop=True)

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,sales
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,101,NaN
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,101,NaN
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,101,NaN
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,101,NaN
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,101,NaN
...,...,...,...,...,...,...,...,...
8591,FOODS_2_395_CA_2_evaluation,FOODS_2_395,FOODS_2,FOODS,CA_2,CA,107,NaN
8592,FOODS_2_396_CA_2_evaluation,FOODS_2_396,FOODS_2,FOODS,CA_2,CA,107,NaN
8593,FOODS_2_397_CA_2_evaluation,FOODS_2_397,FOODS_2,FOODS,CA_2,CA,107,NaN
8594,FOODS_2_398_CA_2_evaluation,FOODS_2_398,FOODS_2,FOODS,CA_2,CA,107,NaN


In [12]:
# sales record + prediction record
sales_df = pd.concat([sales_df, prediction_df])
sales_df = sales_df.reset_index(drop=True)
sales_df

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,sales
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,3.0
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,1,0.0
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,1,0.0
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,1,0.0
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,1,3.0
...,...,...,...,...,...,...,...,...
131391,FOODS_2_395_CA_2_evaluation,FOODS_2_395,FOODS_2,FOODS,CA_2,CA,107,NaN
131392,FOODS_2_396_CA_2_evaluation,FOODS_2_396,FOODS_2,FOODS,CA_2,CA,107,NaN
131393,FOODS_2_397_CA_2_evaluation,FOODS_2_397,FOODS_2,FOODS,CA_2,CA,107,NaN
131394,FOODS_2_398_CA_2_evaluation,FOODS_2_398,FOODS_2,FOODS,CA_2,CA,107,NaN


In [13]:
# make prices data for demo
prices_df = prices_df[prices_df['store_id'].isin(['CA_1', 'CA_2'])].reset_index(drop=True)
prices_df = prices_df[prices_df['item_id'].str.contains('FOODS_1|FOODS_2')].reset_index(drop=True)
prices_df

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,FOODS_1_001,11101,2.0
1,CA_1,FOODS_1_001,11102,2.0
2,CA_1,FOODS_1_001,11103,2.0
3,CA_1,FOODS_1_001,11104,2.0
4,CA_1,FOODS_1_001,11105,2.0
...,...,...,...,...
236875,CA_2,FOODS_2_399,11617,2.0
236876,CA_2,FOODS_2_399,11618,2.0
236877,CA_2,FOODS_2_399,11619,2.0
236878,CA_2,FOODS_2_399,11620,2.0


In [14]:
# save prices csv for demo
prices_df.to_csv('prices_demo.csv', index=False)

In [15]:
# check unique namber of item_id
# 122800(number of sales record) = 2(store_id) * 614(item_id) * 100(date)
len(prices_df['item_id'].unique())

614

In [16]:
# make calendar csv for demo
calendar_df = calendar_df.rename(columns={'d': 'date_id'})
calendar_df['date_id'] = calendar_df['date_id'].apply(lambda x: x[2:]).astype(np.int64)
calendar_df

,date,wm_yr_wk,weekday,wday,month,year,date_id,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,1968,NaN,NaN,NaN,NaN,0,0,0


In [17]:
# save calendar csv for demo
calendar_df.to_csv('calendar_demo.csv', index=False)

In [18]:
# join calendar
sales_df = pd.merge(sales_df, calendar_df[['wm_yr_wk', 'date_id', 'event_name_1', 'event_type_1' ,'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI' ]], on='date_id', how='left'  )
sales_df

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,sales,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,3.0,11101,NaN,NaN,NaN,NaN,0,0,0
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,NaN,NaN,NaN,0,0,0
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,NaN,NaN,NaN,0,0,0
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,NaN,NaN,NaN,0,0,0
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,1,3.0,11101,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131391,FOODS_2_395_CA_2_evaluation,FOODS_2_395,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,NaN,NaN,NaN,0,1,1
131392,FOODS_2_396_CA_2_evaluation,FOODS_2_396,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,NaN,NaN,NaN,0,1,1
131393,FOODS_2_397_CA_2_evaluation,FOODS_2_397,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,NaN,NaN,NaN,0,1,1
131394,FOODS_2_398_CA_2_evaluation,FOODS_2_398,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,NaN,NaN,NaN,0,1,1


In [19]:
# join prices
sales_df = sales_df.merge(prices_df, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
sales_df

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,sales,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,3.0,11101,NaN,NaN,NaN,NaN,0,0,0,2.00
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,NaN,NaN,NaN,0,0,0,7.88
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,NaN,NaN,NaN,0,0,0,2.88
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,NaN,NaN,NaN,0,0,0,NaN
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,1,3.0,11101,NaN,NaN,NaN,NaN,0,0,0,2.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131391,FOODS_2_395_CA_2_evaluation,FOODS_2_395,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,NaN,NaN,NaN,0,1,1,NaN
131392,FOODS_2_396_CA_2_evaluation,FOODS_2_396,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,NaN,NaN,NaN,0,1,1,NaN
131393,FOODS_2_397_CA_2_evaluation,FOODS_2_397,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,NaN,NaN,NaN,0,1,1,NaN
131394,FOODS_2_398_CA_2_evaluation,FOODS_2_398,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,NaN,NaN,NaN,0,1,1,NaN


In [20]:
# check first week of item in store
prices_df.groupby(['store_id', 'item_id'])['wm_yr_wk'].agg(['min'])

min
store_id item_id           
CA_1     FOODS_1_001  11101
         FOODS_1_002  11101
         FOODS_1_003  11101
         FOODS_1_004  11206
         FOODS_1_005  11101
...                     ...
CA_2     FOODS_2_395  11517
         FOODS_2_396  11517
         FOODS_2_397  11517
         FOODS_2_398  11517
         FOODS_2_399  11101

[1228 rows x 1 columns]

In [21]:
# create release_df
release_df = prices_df.groupby(['store_id', 'item_id'])['wm_yr_wk'].agg(['min']).reset_index()
release_df.columns = ['store_id', 'item_id', 'release']
display(release_df)

,store_id,item_id,release
0,CA_1,FOODS_1_001,11101
1,CA_1,FOODS_1_002,11101
2,CA_1,FOODS_1_003,11101
3,CA_1,FOODS_1_004,11206
4,CA_1,FOODS_1_005,11101
...,...,...,...
1223,CA_2,FOODS_2_395,11517
1224,CA_2,FOODS_2_396,11517
1225,CA_2,FOODS_2_397,11517
1226,CA_2,FOODS_2_398,11517


In [22]:
# cerate feature release
release_df['release'] = release_df['release'] - release_df['release'].min()
release_df['release'] = release_df['release'].astype(np.int64)
display(release_df)

,store_id,item_id,release
0,CA_1,FOODS_1_001,0
1,CA_1,FOODS_1_002,0
2,CA_1,FOODS_1_003,0
3,CA_1,FOODS_1_004,105
4,CA_1,FOODS_1_005,0
...,...,...,...
1223,CA_2,FOODS_2_395,416
1224,CA_2,FOODS_2_396,416
1225,CA_2,FOODS_2_397,416
1226,CA_2,FOODS_2_398,416


In [23]:
# join release
sales_df = sales_df.merge(release_df, on=['store_id', 'item_id'], how='left')
display(sales_df)

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,sales,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,release
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,3.0,11101,NaN,NaN,NaN,NaN,0,0,0,2.00,0
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,NaN,NaN,NaN,0,0,0,7.88,0
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,NaN,NaN,NaN,0,0,0,2.88,0
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,NaN,NaN,NaN,0,0,0,NaN,105
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,1,3.0,11101,NaN,NaN,NaN,NaN,0,0,0,2.94,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131391,FOODS_2_395_CA_2_evaluation,FOODS_2_395,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,NaN,NaN,NaN,0,1,1,NaN,416
131392,FOODS_2_396_CA_2_evaluation,FOODS_2_396,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,NaN,NaN,NaN,0,1,1,NaN,416
131393,FOODS_2_397_CA_2_evaluation,FOODS_2_397,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,NaN,NaN,NaN,0,1,1,NaN,416
131394,FOODS_2_398_CA_2_evaluation,FOODS_2_398,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,NaN,NaN,NaN,0,1,1,NaN,416


In [24]:
# price statistics feature
sales_df['price_max'] = sales_df.groupby(['store_id', 'item_id'])['sell_price'].transform('max')
sales_df['price_min'] = sales_df.groupby(['store_id', 'item_id'])['sell_price'].transform('min')
sales_df['price_std'] = sales_df.groupby(['store_id', 'item_id'])['sell_price'].transform('std')
sales_df['price_mean'] = sales_df.groupby(['store_id', 'item_id'])['sell_price'].transform('mean')
sales_df['price_norm'] = sales_df['sell_price'] / sales_df['price_max']
sales_df['price_nunique'] = sales_df.groupby(['store_id', 'item_id'])['sell_price'].transform('nunique')
sales_df['item_nunique'] = sales_df.groupby(['store_id', 'sell_price'])['item_id'].transform('nunique')
display(sales_df)

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,sales,wm_yr_wk,event_name_1,...,snap_WI,sell_price,release,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,3.0,11101,NaN,...,0,2.00,0,2.00,2.00,0.0,2.00,1.0,1,20.0
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,...,0,7.88,0,7.88,7.88,0.0,7.88,1.0,1,1.0
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,...,0,2.88,0,2.88,2.88,0.0,2.88,1.0,1,13.0
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,...,0,NaN,105,NaN,NaN,NaN,NaN,NaN,0,NaN
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,1,3.0,11101,NaN,...,0,2.94,0,2.94,2.94,0.0,2.94,1.0,1,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131391,FOODS_2_395_CA_2_evaluation,FOODS_2_395,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,...,1,NaN,416,NaN,NaN,NaN,NaN,NaN,0,NaN
131392,FOODS_2_396_CA_2_evaluation,FOODS_2_396,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,...,1,NaN,416,NaN,NaN,NaN,NaN,NaN,0,NaN
131393,FOODS_2_397_CA_2_evaluation,FOODS_2_397,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,...,1,NaN,416,NaN,NaN,NaN,NaN,NaN,0,NaN
131394,FOODS_2_398_CA_2_evaluation,FOODS_2_398,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,...,1,NaN,416,NaN,NaN,NaN,NaN,NaN,0,NaN


In [25]:
# note price feature
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131396 entries, 0 to 131395
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             131396 non-null  object 
 1   item_id        131396 non-null  object 
 2   dept_id        131396 non-null  object 
 3   cat_id         131396 non-null  object 
 4   store_id       131396 non-null  object 
 5   state_id       131396 non-null  object 
 6   date_id        131396 non-null  int64  
 7   sales          122800 non-null  float64
 8   wm_yr_wk       131396 non-null  int64  
 9   event_name_1   13508 non-null   object 
 10  event_type_1   13508 non-null   object 
 11  event_name_2   1228 non-null    object 
 12  event_type_2   1228 non-null    object 
 13  snap_CA        131396 non-null  int64  
 14  snap_TX        131396 non-null  int64  
 15  snap_WI        131396 non-null  int64  
 16  sell_price     51484 non-null   float64
 17  release        131396 non-nul

In [26]:
# sales lag feature
lag_df = sales_df[['id', 'date_id', 'sales']]

num_lag_day_list = []
num_lag_day = 15
for col in range(predict_horizon, predict_horizon + num_lag_day):
    num_lag_day_list.append(col)

num_rolling_day_list = [7, 14, 30, 60]
num_shift_rolling_day_list = []
for num_shift_day in [1, 7, 14]:
    for num_rolling_day in [7, 14, 30]:
        num_shift_rolling_day_list.append([num_shift_day, num_rolling_day])

lag_df = lag_df.assign(**{
    'lag_{}_{}'.format(col, l): lag_df.groupby(['id'])['sales'].transform(lambda x: x.shift(l))
    for l in num_lag_day_list
})

for col in list(lag_df):
    if 'lag' in col:
        lag_df[col] = lag_df[col]

for num_rolling_day in num_rolling_day_list:
    lag_df['rolling_mean_' + str(num_rolling_day)] = lag_df.groupby(['id'])['sales'].transform(
        lambda x: x.shift(predict_horizon).rolling(num_rolling_day).mean())
    lag_df['rolling_std_' + str(num_rolling_day)] = lag_df.groupby(['id'])['sales'].transform(
        lambda x: x.shift(predict_horizon).rolling(num_rolling_day).std())

display(lag_df)

,id,date_id,sales,lag_21_7,lag_21_8,lag_21_9,lag_21_10,lag_21_11,lag_21_12,lag_21_13,...,lag_21_20,lag_21_21,rolling_mean_7,rolling_std_7,rolling_mean_14,rolling_std_14,rolling_mean_30,rolling_std_30,rolling_mean_60,rolling_std_60
0,FOODS_1_001_CA_1_evaluation,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FOODS_1_002_CA_1_evaluation,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FOODS_1_003_CA_1_evaluation,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FOODS_1_004_CA_1_evaluation,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FOODS_1_005_CA_1_evaluation,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131391,FOODS_2_395_CA_2_evaluation,107,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
131392,FOODS_2_396_CA_2_evaluation,107,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
131393,FOODS_2_397_CA_2_evaluation,107,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
131394,FOODS_2_398_CA_2_evaluation,107,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [27]:
# add columns of lag feature
sales_df = pd.concat([sales_df, lag_df.iloc[:, 3:]], axis=1)
display(sales_df)

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,sales,wm_yr_wk,event_name_1,...,lag_21_20,lag_21_21,rolling_mean_7,rolling_std_7,rolling_mean_14,rolling_std_14,rolling_mean_30,rolling_std_30,rolling_mean_60,rolling_std_60
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,3.0,11101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,1,3.0,11101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131391,FOODS_2_395_CA_2_evaluation,FOODS_2_395,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
131392,FOODS_2_396_CA_2_evaluation,FOODS_2_396,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
131393,FOODS_2_397_CA_2_evaluation,FOODS_2_397,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
131394,FOODS_2_398_CA_2_evaluation,FOODS_2_398,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [28]:
# note lag feature
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131396 entries, 0 to 131395
Data columns (total 48 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               131396 non-null  object 
 1   item_id          131396 non-null  object 
 2   dept_id          131396 non-null  object 
 3   cat_id           131396 non-null  object 
 4   store_id         131396 non-null  object 
 5   state_id         131396 non-null  object 
 6   date_id          131396 non-null  int64  
 7   sales            122800 non-null  float64
 8   wm_yr_wk         131396 non-null  int64  
 9   event_name_1     13508 non-null   object 
 10  event_type_1     13508 non-null   object 
 11  event_name_2     1228 non-null    object 
 12  event_type_2     1228 non-null    object 
 13  snap_CA          131396 non-null  int64  
 14  snap_TX          131396 non-null  int64  
 15  snap_WI          131396 non-null  int64  
 16  sell_price       51484 non-null   floa

In [29]:
# mask with date
train_mask = sales_df['date_id'] <= (end_train_day_x - predict_horizon)
valid_mask = ((end_train_day_x - predict_horizon) < sales_df['date_id']) & (sales_df['date_id'] <= end_train_day_x)
preds_mask = end_train_day_x < sales_df['date_id']

In [30]:
# training
sales_df[train_mask]

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,sales,wm_yr_wk,event_name_1,...,lag_21_20,lag_21_21,rolling_mean_7,rolling_std_7,rolling_mean_14,rolling_std_14,rolling_mean_30,rolling_std_30,rolling_mean_60,rolling_std_60
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,3.0,11101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,1,0.0,11101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,1,3.0,11101,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114199,FOODS_2_395_CA_2_evaluation,FOODS_2_395,FOODS_2,FOODS,CA_2,CA,93,0.0,11114,NaN,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
114200,FOODS_2_396_CA_2_evaluation,FOODS_2_396,FOODS_2,FOODS,CA_2,CA,93,0.0,11114,NaN,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
114201,FOODS_2_397_CA_2_evaluation,FOODS_2_397,FOODS_2,FOODS,CA_2,CA,93,0.0,11114,NaN,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
114202,FOODS_2_398_CA_2_evaluation,FOODS_2_398,FOODS_2,FOODS,CA_2,CA,93,0.0,11114,NaN,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [31]:
# validation
sales_df[valid_mask]

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,sales,wm_yr_wk,event_name_1,...,lag_21_20,lag_21_21,rolling_mean_7,rolling_std_7,rolling_mean_14,rolling_std_14,rolling_mean_30,rolling_std_30,rolling_mean_60,rolling_std_60
114204,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,94,7.0,11114,NaN,...,1.0,2.0,0.428571,0.786796,0.714286,0.825420,0.766667,1.006302,1.133333,1.383519
114205,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,94,2.0,11114,NaN,...,0.0,0.0,0.857143,1.069045,0.714286,1.266647,0.566667,1.006302,0.483333,0.873172
114206,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,94,3.0,11114,NaN,...,1.0,1.0,0.714286,1.112697,0.928571,1.141139,1.000000,1.114172,0.900000,1.036618
114207,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,94,0.0,11114,NaN,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
114208,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,94,1.0,11114,NaN,...,1.0,2.0,7.285714,9.013218,4.714286,6.832373,3.100000,4.901442,2.383333,3.682720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122795,FOODS_2_395_CA_2_evaluation,FOODS_2_395,FOODS_2,FOODS,CA_2,CA,100,0.0,11115,Mother's day,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
122796,FOODS_2_396_CA_2_evaluation,FOODS_2_396,FOODS_2,FOODS,CA_2,CA,100,0.0,11115,Mother's day,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
122797,FOODS_2_397_CA_2_evaluation,FOODS_2_397,FOODS_2,FOODS,CA_2,CA,100,0.0,11115,Mother's day,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
122798,FOODS_2_398_CA_2_evaluation,FOODS_2_398,FOODS_2,FOODS,CA_2,CA,100,0.0,11115,Mother's day,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [32]:
# prediction
sales_df[preds_mask]

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,sales,wm_yr_wk,event_name_1,...,lag_21_20,lag_21_21,rolling_mean_7,rolling_std_7,rolling_mean_14,rolling_std_14,rolling_mean_30,rolling_std_30,rolling_mean_60,rolling_std_60
122800,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,101,NaN,11115,NaN,...,2.0,0.0,1.714286,2.627691,1.071429,1.979288,1.066667,1.552158,1.033333,1.365019
122801,FOODS_1_002_CA_1_evaluation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,101,NaN,11115,NaN,...,0.0,0.0,1.285714,0.951190,1.071429,0.997249,0.766667,1.072648,0.583333,0.944057
122802,FOODS_1_003_CA_1_evaluation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,101,NaN,11115,NaN,...,1.0,0.0,1.428571,0.975900,1.071429,1.071612,1.000000,1.082781,1.000000,1.073565
122803,FOODS_1_004_CA_1_evaluation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,101,NaN,11115,NaN,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
122804,FOODS_1_005_CA_1_evaluation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,101,NaN,11115,NaN,...,2.0,6.0,1.428571,1.272418,4.357143,6.890430,3.100000,4.922503,2.366667,3.677539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131391,FOODS_2_395_CA_2_evaluation,FOODS_2_395,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
131392,FOODS_2_396_CA_2_evaluation,FOODS_2_396,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
131393,FOODS_2_397_CA_2_evaluation,FOODS_2_397,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
131394,FOODS_2_398_CA_2_evaluation,FOODS_2_398,FOODS_2,FOODS,CA_2,CA,107,NaN,11116,NaN,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [39]:
# train with lightgbm
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'rmse',
        'metric': 'rmse',
        'learning_rate': 0.03,
        'num_leaves': 31,
        'min_data_in_leaf': 20,
        'feature_fraction': 1.0,
        'max_bin': 255,
        'num_boost_round': 1000,
        'random_state': 42,
        'verbose': -1,
    }


store_id_set_list = list(sales_df['store_id'].unique())
print(f"training stores: {store_id_set_list}")

feature_importance_all_df = pd.DataFrame()
for store_index, store_id in enumerate(store_id_set_list):
    print(f'now training {store_id} store')

    train_df=sales_df[train_mask].reset_index(drop=True)
    train_store_df=train_df[train_df['store_id'] == store_id].reset_index(drop=True)
    valid_df=sales_df[valid_mask].reset_index(drop=True)
    valid_store_df=valid_df[valid_df['store_id'] == store_id].reset_index(drop=True)
    remove_features = ['id', 'cat_id', 'state_id', 'store_id', 'wm_yr_wk', 'date_id', 'sales']
    enable_features = [col for col in list(sales_df) if col not in remove_features]

    index_columns = ['item_id', 'dept_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    for col in index_columns:
      train_store_df[col] = train_store_df[col].astype('category')
      valid_store_df[col] = valid_store_df[col].astype('category')


    train_data = lgb.Dataset(train_store_df[enable_features].reset_index(drop=True),
                              label=train_store_df['sales'].reset_index(drop=True))

    valid_data = lgb.Dataset(valid_store_df[enable_features].reset_index(drop=True),
                              label=valid_store_df['sales'].reset_index(drop=True))

    estimator = lgb.train(
                          lgb_params,
                          train_data,
                          valid_sets=[train_data, valid_data],
                          valid_names=['train', 'valid'],
                          callbacks=[lgb.early_stopping(10), lgb.log_evaluation(period=10, show_stdv=False)],
                          )
    model_name = str(f'lgb_model_{store_id}_{predict_horizon}_m5.bin')

    feature_importance_store_df = pd.DataFrame(sorted(zip(enable_features, estimator.feature_importance())),
                                                columns=['feature_name', 'importance'])
    feature_importance_store_df = feature_importance_store_df.sort_values('importance', ascending=False)
    feature_importance_store_df['store_id'] = store_id
    feature_importance_store_df.to_csv(f'feature_importance_{store_id}_{predict_horizon}_m5.csv', index=False)
    feature_importance_all_df = pd.concat([feature_importance_all_df, feature_importance_store_df])
    pickle.dump(estimator, open(model_name, 'wb'))

feature_importance_all_df.to_csv(f'feature_importance_all_{predict_horizon}_m5.csv', index=False)
feature_importance_agg_df = feature_importance_all_df.groupby(
    'feature_name')['importance'].agg(['mean', 'std']).reset_index()
feature_importance_agg_df.columns = ['feature_name', 'importance_mean', 'importance_std']
feature_importance_agg_df = feature_importance_agg_df.sort_values('importance_mean', ascending=False)
feature_importance_agg_df.to_csv(f'feature_importance_agg_{predict_horizon}_m5.csv', index=False)


training stores: ['CA_1', 'CA_2']
now training CA_1 store


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[10]	train's rmse: 2.55306	valid's rmse: 2.21054
[20]	train's rmse: 2.22989	valid's rmse: 2.04469
[30]	train's rmse: 2.01892	valid's rmse: 1.97293
[40]	train's rmse: 1.87904	valid's rmse: 1.93628
[50]	train's rmse: 1.78507	valid's rmse: 1.92779
[60]	train's rmse: 1.71871	valid's rmse: 1.92623
Early stopping, best iteration is:
[58]	train's rmse: 1.7303	valid's rmse: 1.92273
now training CA_2 store


/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[10]	train's rmse: 2.03766	valid's rmse: 1.80688
[20]	train's rmse: 1.82885	valid's rmse: 1.64398
[30]	train's rmse: 1.69378	valid's rmse: 1.55019
[40]	train's rmse: 1.60588	valid's rmse: 1.49723
[50]	train's rmse: 1.54463	valid's rmse: 1.46742
[60]	train's rmse: 1.50182	valid's rmse: 1.45703
[70]	train's rmse: 1.4683	valid's rmse: 1.44984
[80]	train's rmse: 1.44168	valid's rmse: 1.44804
Early stopping, best iteration is:
[75]	train's rmse: 1.45442	valid's rmse: 1.44756


In [40]:
# predict with lightgbm
store_id_set_list = ['CA_1', 'CA_2']
end_train_day_x_list = [100]
prediction_horizon_list = [7]

pred_v_all_df = list()

for end_train_day_x in end_train_day_x_list:
    previous_prediction_horizon = 0
    for prediction_horizon in prediction_horizon_list:

        pred_v_df = pd.DataFrame()

        for store_index, store_id in enumerate(store_id_set_list):

            model_path = str(f'./lgb_model_{store_id}_{prediction_horizon}_m5.bin')
            print(f'loading {model_path}')
            estimator = pickle.load(open(model_path, 'rb'))

            preds_df=sales_df[preds_mask].reset_index(drop=True)
            preds_store_df=preds_df[preds_df['store_id'] == store_id].reset_index(drop=True)

            index_columns = ['item_id', 'dept_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
            for col in index_columns:
              preds_store_df[col] = preds_store_df[col].astype('category')


            for predict_day in range(previous_prediction_horizon + 1, prediction_horizon + 1):
                print('[{3} -> {4}] predict {0}/{1} {2} day {5}'.format(
                store_index + 1, len(store_id_set_list), store_id,
                previous_prediction_horizon + 1, prediction_horizon, predict_day))
                mask = preds_store_df['date_id'] == (end_train_day_x + predict_day)
                preds_store_df.loc[mask, 'sales'] = estimator.predict(preds_store_df[mask][enable_features])

            temp_v_df = preds_store_df[
                    (preds_store_df['date_id'] >= end_train_day_x + previous_prediction_horizon + 1) &
                    (preds_store_df['date_id'] < end_train_day_x + prediction_horizon + 1)
                    ][['id', 'date_id', 'sales']]

            if len(pred_v_df)!=0:
                pred_v_df = pd.concat([pred_v_df, temp_v_df])
            else:
                pred_v_df = temp_v_df.copy()

        previous_prediction_horizon = prediction_horizon

        pred_v_all_df.append(pred_v_df)

pred_v_all_df = pd.concat(pred_v_all_df)

loading ./lgb_model_CA_1_7_m5.bin
[1 -> 7] predict 1/2 CA_1 day 1
[1 -> 7] predict 1/2 CA_1 day 2
[1 -> 7] predict 1/2 CA_1 day 3
[1 -> 7] predict 1/2 CA_1 day 4
[1 -> 7] predict 1/2 CA_1 day 5
[1 -> 7] predict 1/2 CA_1 day 6
[1 -> 7] predict 1/2 CA_1 day 7
loading ./lgb_model_CA_2_7_m5.bin
[1 -> 7] predict 2/2 CA_2 day 1
[1 -> 7] predict 2/2 CA_2 day 2
[1 -> 7] predict 2/2 CA_2 day 3
[1 -> 7] predict 2/2 CA_2 day 4
[1 -> 7] predict 2/2 CA_2 day 5
[1 -> 7] predict 2/2 CA_2 day 6
[1 -> 7] predict 2/2 CA_2 day 7


In [41]:
# prediction
pred_v_all_df=pred_v_all_df.reset_index(drop=True)
pred_v_all_df

,id,date_id,sales
0,FOODS_1_001_CA_1_evaluation,101,1.198667
1,FOODS_1_002_CA_1_evaluation,101,0.900342
2,FOODS_1_003_CA_1_evaluation,101,1.139522
3,FOODS_1_004_CA_1_evaluation,101,0.189177
4,FOODS_1_005_CA_1_evaluation,101,2.062190
...,...,...,...
8591,FOODS_2_395_CA_2_evaluation,107,0.077215
8592,FOODS_2_396_CA_2_evaluation,107,0.077215
8593,FOODS_2_397_CA_2_evaluation,107,0.077215
8594,FOODS_2_398_CA_2_evaluation,107,0.077215
